# 심층 신경망

## 1) 패션 데이터 불러오기
- 28*28의 2차원 데이터를 1차원 784 픽셀 데이터로 변경
- 변경 뒤 해당 데이터를 훈련 / 테스트 세트로 나눔

In [1]:
import keras
(train_input, train_target),(test_input,test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
from sklearn.model_selection import train_test_split

### 0~255 인데이터를 0~1 데이터로 변경
train_scaled = train_input / 255.0

### 28*28 > 784
train_scaled = train_scaled.reshape(-1,28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

## 은닉층
- 입력층과 출력층 사이 밀집층이 추가된 층
- `활성화 함수` : 신경망 층의 선형 방정식의 계산 값에 적용하는 함수
- 출력층에 적용되는 활성화 함수는 제한됨
  - 이진 분류 : 시그모이드
  - 다중 분류 : 소프트맥스
- 하지만 은닉층의 활성화 함수는 비교적 자유로움 (시그모이드함수, 렐루 함수 등..)

### 은닉층에 활성화 함수가 적용되는 이유
- 은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없어짐
- 따라서 비선형함수로 변경하여 역할이 없어지지 않도록 함

## 은닉층 + 출력층 만들기
- 은닉층 : 시그모이드 활성화 함수 / 뉴런 100개  
- 출력층 : 소프트맥스 함수 / 뉴런 10개
- 뉴런 갯수의 기준은 따로 없으나, **은닉층이 출력층보다 많아야함**

In [3]:
inputs = keras.layers.Input(shape=(784,))

## 은닉층
dense1 = keras.layers.Dense(100, activation="sigmoid")
### 출력층
dense2 = keras.layers.Dense(10,activation="softmax")

## 3) 심층 신경망 만들기 (DNN)
- `keras.Sequential([입력층, 은닉층, 출력층])`

In [4]:
model = keras.Sequential([inputs, dense1, dense2])

- 층에 대한 정보 확인
  - 출력크기 (None, 100) :
  - None : 케라스 모델의 fit()은 데이터를 주입하면 잘게 나누어 여러번 경사하강법 사용
    --> 미니 배치 경사하강법 사용
  - 기본적인 미니배치 크기 32
    --> `batch_size`  옵션을 이용해 바꿀 수 있음
  - 따라서 샘플 개수를 고정하지 않고 None으로 선정
  - 100 : 은닉층 유닛 100개
    - 784개의 특성이 100개의 유닛으로 압축됨

- 출력층의 파라미터 수 : 1010개
  - 은닉층 유닛 100개
  - 출력층 유닉 10개
  - 은닉층 - 출력층 조합 : 1000개
  - 출력층의 절편 10개
  - 총 1010개

- 특성 하나당 1010개 * 특성 총 784개

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

### 생성자를 이용해 층 추가
- 생성자에서 바로 클래스를 만들 수도 있음

In [6]:
model = keras.Sequential([
keras.layers.Input(shape=(784,)),
keras.layers.Dense(100, activation="sigmoid", name="은닉층"),
keras.layers.Dense(10, activation="softmax",name="출력층")
], name="패션 MNIST 모델")

In [7]:
model.summary()

Model: "패션 MNIST 모델"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 은닉층 (Dense)                  │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 출력층 (Dense)                  │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

### `add()`를 이용해 층 추가

In [8]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(784,)))
model.add(keras.layers.Dense(100,activation="sigmoid"))
model.add(keras.layers.Dense(10,activation="softmax"))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

### 학습 진행
- 87% 까지 정확도가 올라옴

In [9]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7561 - loss: 0.7578
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8509 - loss: 0.4125
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8629 - loss: 0.3844
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8721 - loss: 0.3506
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8771 - loss: 0.3400


# 렐루 함수
## 시그모이드 함수의 단점
- 오른쪽과 왼쪽끝으로 갈수록 그래프가 누워 있어 올바른 출력을 만드는데 신속하게 대응하지 못함
- 신경망이 많을수록 누적되어 더 여려워짐

## 렐루(ReLU) 함수
- 표현 $max(0,z)$
- 입력이 양수 = 입력값 그대로 통과
- 입력이 음수 = 0

### Flatten 층
- 배치 차원을 제외한 나머지 입력차원을 모두 일렬로 만듦
- 가중치나 절편이 없기 때문에 성능에 기여하는 바 X
- 하지만 입력층과 은닉층 사이에 추가하기 때문에 층이라고 부름
  - flatten_1 는 Param = 0 임 
- Flatten층을 신경망 모델에 추가하면 입력값의 차원을 알 수 있음 (None, 784)    

In [13]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100,activation="sigmoid"))
model.add(keras.layers.Dense(10,activation="softmax"))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

- 모델 다시 훈련 
  - 시그모이드에 비해 성능이 약간의 향상 

In [16]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_scaled, train_target,epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7451 - loss: 0.7901
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8489 - loss: 0.4202
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8631 - loss: 0.3830
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8700 - loss: 0.3567
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8777 - loss: 0.3392


- 검증세트에서의 정확도

In [17]:
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8690 - loss: 0.3620


[0.3642233610153198, 0.8662499785423279]

## 옵티마이저
### 신경망에서의 하이퍼 파라미터 
  - 은닉층 개수
 - 뉴런 개수 
- 활성화 함수
-  epochs

## 옵티마이저 
- 케라스에서 사용하는 다양한 종류의 경사 하강법 알고리즘 

#### SGD의 기본 경사 하강 옵티마이저 
learning_rate -> momentum > 0 -> nesterov=True
RMSprop -> Adam
Adagrad

In [22]:
sgd = keras.optimizers.SGD(learning_rate=0.01,momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,loss="sparse_categorical_crossentropy", metrics=["accuracy"])


- Adam을 사용한 경우

In [23]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam")
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8840 - loss: 0.3172
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8888 - loss: 0.3065
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8927 - loss: 0.2920
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8978 - loss: 0.2790
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9000 - loss: 0.2727


In [25]:
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8829 - loss: 0.3274


[0.32827767729759216, 0.8803333044052124]

----